In [3]:
import anthropic
import wikipedia
from dotenv import load_dotenv
load_dotenv()

client = anthropic.Anthropic()

In [4]:
def get_article(search_term):
    results = wikipedia.search(search_term)
    first_result = results[0]
    page = wikipedia.page(first_result, auto_suggest=False)
    return page.content

In [5]:
get_article("Tamerlane")[:500]

'Timur, also known as Tamerlane (8 April 1336 – 17–18 February 1405), was a Turco-Mongol conqueror who founded the Timurid Empire in and around modern-day Afghanistan, Iran, and Central Asia, becoming the first ruler of the Timurid dynasty. An undefeated commander, he is widely regarded as one of the greatest military leaders and tacticians in history, as well as one of the most brutal and deadly. Timur is also considered a great patron of art and architecture as he interacted with intellectuals '

In [20]:
wikipedia_search_schema = {
    "name": "get_article",
    "description": "A tool to retrieve an up to date Wikipedia article.",
    "input_schema": {
        "type": "object",
        "properties": {"search_term": {"type": "string", "description": "The search term to search for in wikipedia"}},
        "required": ["search_term"],
    },

}

In [7]:
TOOL_MAP = {"get_article": get_article}

In [48]:
def sanitize_message(message) -> dict:
    message_dict = message.to_dict()
    for k in ["id", "model", "stop_reason", "stop_sequence", "type", "usage"]:
        del message_dict[k]
    return message_dict


def sample_with_tools(**claude_kwargs) -> list[dict]:
    messages = claude_kwargs["messages"]
    curr_res = client.messages.create(**claude_kwargs)
    while curr_res.stop_reason == "tool_use":
        tool_blocks = [c for c in curr_res.content if isinstance(c, anthropic.types.tool_use_block.ToolUseBlock)]
        tool_message = {"role": "user", "content": []}
        messages.append(sanitize_message(curr_res))
        for tb in tool_blocks:
            tool_output = TOOL_MAP[tb.name](**tb.input)
            tool_message["content"].append(
                {
                    "type": "tool_result",
                    "tool_use_id": tb.id,
                    "content": tool_output,
                }
            )
        messages.append(tool_message)
        claude_kwargs["messages"] = messages
        curr_res = client.messages.create(**claude_kwargs)
    messages.append(sanitize_message(curr_res))
    return messages
        

In [49]:
response = sample_with_tools(
    model="claude-3-sonnet-20240229",
    messages=[{"role": "user", "content": "Who won the 2024 Masters tournament?"}],
    max_tokens=1000,
    tools=[wikipedia_search_schema],
)

In [50]:
response

[{'role': 'user', 'content': 'Who won the 2024 Masters tournament?'},
 {'content': [{'text': 'Okay, let me search Wikipedia for information on the 2024 Masters tournament winner:',
    'type': 'text'},
   {'id': 'toolu_01FRa8C3CdzdhcV781i4fP92',
    'input': {'search_term': '2024 Masters Tournament'},
    'name': 'get_article',
    'type': 'tool_use'}],
  'role': 'assistant'},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01FRa8C3CdzdhcV781i4fP92',
    'content': "The 2024 Masters Tournament was the 88th edition of the Masters Tournament and the first of the men's four major golf championships held in 2024. The tournament was played from April 11–14 at Augusta National Golf Club in Augusta, Georgia, United States. Scottie Scheffler won his second Masters and major, four strokes ahead of runner-up Ludvig Åberg, who was playing in his first major. Scheffler became the fourth-youngest two-time winner of the tournament and the second player, after Tiger 

In [55]:
response = sample_with_tools(
    model="claude-3-sonnet-20240229",
    messages=[{"role": "user", "content": "Who is the current prime minister of Bulgaria?"}],
    max_tokens=1000,
    tools=[wikipedia_search_schema],
)

In [56]:
response

[{'role': 'user', 'content': 'Who is the current prime minister of Bulgaria?'},
 {'content': [{'text': 'Okay, let me use the Wikipedia tool to find information on the current prime minister of Bulgaria:',
    'type': 'text'},
   {'id': 'toolu_01MWFN3rxVhGng2EoW2qn2ik',
    'input': {'search_term': 'Prime Minister of Bulgaria'},
    'name': 'get_article',
    'type': 'tool_use'}],
  'role': 'assistant'},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01MWFN3rxVhGng2EoW2qn2ik',
    'content': 'The prime minister of Bulgaria (Bulgarian: Министър-председател, romanized: Ministar-predsedatel) is the head of government of Bulgaria. They are the leader of a political coalition in the Bulgarian parliament – known as the National Assembly of Bulgaria (Народно събрание, Narodno sabranie) – and the leader of the cabinet. At times, the prime minister has been appointed by the President of Bulgaria.\nDimitar Glavchev has been the caretaker prime minister of Bulgar

In [57]:
response = sample_with_tools(
    model="claude-3-sonnet-20240229",
    messages=[{"role": "user", "content": "Who won the 2024 F1 Australian Grand Prix?"}],
    max_tokens=1000,
    tools=[wikipedia_search_schema],
)

In [58]:
response

[{'role': 'user', 'content': 'Who won the 2024 F1 Australian Grand Prix?'},
 {'content': [{'text': 'Okay, let me search Wikipedia for information about the 2024 F1 Australian Grand Prix winner.',
    'type': 'text'},
   {'id': 'toolu_01VSVj4bLzFb4eskdzMCtPvj',
    'input': {'search_term': '2024 F1 Australian Grand Prix'},
    'name': 'get_article',
    'type': 'tool_use'}],
  'role': 'assistant'},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01VSVj4bLzFb4eskdzMCtPvj',
    'content': 'The 2024 Australian Grand Prix (officially known as the Formula 1 Rolex Australian Grand Prix 2024) was a Formula One motor race held on 24 March 2024 at the Albert Park Circuit in Melbourne, Victoria. It was the third round of the 2024 Formula One World Championship.\nAfter qualifying second, Ferrari\'s Carlos Sainz Jr. won the race, two weeks after undergoing an appendectomy. Pole-sitter Max Verstappen retired on lap four, breaking his streak of nine consecutive wins.

In [59]:
sample_with_tools(
    model="claude-3-sonnet-20240229",
    messages=[{"role": "user", "content": "Who stars in the movie Challengers?"}],
    max_tokens=1000,
    tools=[wikipedia_search_schema],
)

[{'role': 'user', 'content': 'Who stars in the movie Challengers?'},
 {'content': [{'text': 'Okay, let me look up information about the movie Challengers on Wikipedia:',
    'type': 'text'},
   {'id': 'toolu_012TCNRKJMitHWjoMmCeUpUK',
    'input': {'search_term': 'Challengers (film)'},
    'name': 'get_article',
    'type': 'tool_use'}],
  'role': 'assistant'},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_012TCNRKJMitHWjoMmCeUpUK',
    'content': '}}\nChallengers is a 2024 American romantic sports drama film directed by Luca Guadagnino from a screenplay by Justin Kuritzkes, and stars Zendaya, Josh O\'Connor, and Mike Faist. The film centers around the love triangle between an injured tennis star turned coach (Zendaya), her low-circuit tennis player ex-boyfriend (O’Connor), and her tennis champion husband (Faist) across thirteen years of their shared relationship, culminating in the latter two’s match at the ATP Challenger Tour.\nKuritzkes\' screenpl

In [73]:
system_prompt = """
    You will be asked a question by the user. 
    If answering the question requires data you were not trained on, you can use the get_article tool to get the contents of a recent wikipedia article about the topic. 
    If you can answer the question without needing to get more information, please do so. 
    Only call the tool when needed.
    Keep your final answer as succinct and concise as possible.
"""
sample_with_tools(
    model="claude-3-sonnet-20240229",
    system=system_prompt,
    messages=[{"role": "user", "content": "Who wrote the book life of pi?"}],
    max_tokens=1000,
    tools=[wikipedia_search_schema],
)

[{'role': 'user', 'content': 'Who wrote the book life of pi?'},
 {'content': [{'text': 'The book "Life of Pi" was written by the Canadian author Yann Martel. It was published in 2001 and won the prestigious Man Booker Prize in 2002. The novel tells the story of an Indian boy named Pi who survives a shipwreck and is stranded on a lifeboat in the Pacific Ocean with a Bengal tiger named Richard Parker. "Life of Pi" explores themes of spirituality, survival, and man\'s connection to nature through Pi\'s extraordinary journey.',
    'type': 'text'}],
  'role': 'assistant'}]

In [74]:
sample_with_tools(
    model="claude-3-sonnet-20240229",
    system=system_prompt,
    messages=[{"role": "user", "content": "Who wrote the score for the movie Challengers?"}],
    max_tokens=1000,
    tools=[wikipedia_search_schema],
)

[{'role': 'user',
  'content': 'Who wrote the score for the movie Challengers???'},
 {'content': [{'id': 'toolu_01MeJ86WZoLhQPsBWjfQG9Ks',
    'input': {'search_term': 'Challengers (film)'},
    'name': 'get_article',
    'type': 'tool_use'}],
  'role': 'assistant'},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01MeJ86WZoLhQPsBWjfQG9Ks',
    'content': '}}\nChallengers is a 2024 American romantic sports drama film directed by Luca Guadagnino from a screenplay by Justin Kuritzkes, and stars Zendaya, Josh O\'Connor, and Mike Faist. The film centers around the love triangle between an injured tennis star turned coach (Zendaya), her low-circuit tennis player ex-boyfriend (O’Connor), and her tennis champion husband (Faist) across thirteen years of their shared relationship, culminating in the latter two’s match at the ATP Challenger Tour.\nKuritzkes\' screenplay was acquired by MGM in February 2022, with the main cast and Guadagnino hired shortly after; 

In [75]:
sample_with_tools(
    model="claude-3-sonnet-20240229",
    system=system_prompt,
    messages=[{"role": "user", "content": "How many legs does an octopus have?"}],
    max_tokens=1000,
    tools=[wikipedia_search_schema],
)

[{'role': 'user', 'content': 'How many legs does an octopus have?'},
 {'content': [{'text': 'An octopus has eight legs.', 'type': 'text'}],
  'role': 'assistant'}]

In [76]:
sample_with_tools(
    model="claude-3-sonnet-20240229",
    system=system_prompt,
    messages=[{"role": "user", "content": "How many Oscars does Christopher Nolan have? Does he have more than the number of Emmy's that Ben Stiller has?"}],
    max_tokens=1000,
    tools=[wikipedia_search_schema],
)

[{'role': 'user',
  'content': "How many Oscars does Christopher Nolan have? Does he have more than the number of Emmy's that Ben Stiller has?"},
 {'content': [{'id': 'toolu_01NSNeX5u5GYnh329roTJUwT',
    'input': {'search_term': 'Christopher Nolan'},
    'name': 'get_article',
    'type': 'tool_use'}],
  'role': 'assistant'},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01NSNeX5u5GYnh329roTJUwT',
    'content': 'Sir Christopher Edward Nolan  (born 30 July 1970) is a British and American filmmaker. Known for his Hollywood blockbusters with complex storytelling, he is considered a leading filmmaker of the 21st century. Nolan\'s films have earned over $6 billion worldwide, making him the seventh-highest-grossing film director of all time. His accolades include two Academy Awards and two British Academy Film Awards. Nolan was appointed as a Commander of the Order of the British Empire in 2019, and received a knighthood in 2024 for his contributions to 

In [77]:
sample_with_tools(
    model="claude-3-sonnet-20240229",
    system=system_prompt,
    messages=[{"role": "user", "content": "How much bigger was the British Empire at its peak compared to the Mongol Empire at its peak?"}],
    max_tokens=1000,
    tools=[wikipedia_search_schema],
)

[{'role': 'user',
  'content': 'How much bigger was the British Empire at its peak compared to the Mongol Empire at its peak?'},
 {'content': [{'id': 'toolu_01MXFxXxWppZeo8RHPmTLezk',
    'input': {'search_term': 'British Empire'},
    'name': 'get_article',
    'type': 'tool_use'}],
  'role': 'assistant'},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01MXFxXxWppZeo8RHPmTLezk',
    'content': 'The British Empire comprised the dominions, colonies, protectorates, mandates, and other territories ruled or administered by the United Kingdom and its predecessor states. It began with the overseas possessions and trading posts established by England in the late 16th and early 17th centuries. At its height in the 19th and early 20th century, it was the largest empire in history and, for a century, was the foremost global power. By 1913, the British Empire held sway over 412 million people, 23 percent of the world population at the time, and by 1920, it cover

In [79]:
sample_with_tools(
    model="claude-3-sonnet-20240229",
    # system=system_prompt,
    messages=[{"role": "user", "content": "How much bigger was the British Empire at its peak compared to the Mongol Empire at its peak?"}],
    max_tokens=1000,
    # tools=[wikipedia_search_schema],
)

[{'role': 'user',
  'content': 'How much bigger was the British Empire at its peak compared to the Mongol Empire at its peak?'},
 {'content': [{'text': "The British Empire was significantly larger than the Mongol Empire at their respective peaks.\n\nThe Mongol Empire reached its greatest extent in the 13th century under Kublai Khan. At its peak, it covered around 33 million square km (12 million square miles) across Asia and parts of eastern Europe.\n\nThe British Empire reached its territorial peak in the early 20th century, covering around 33.8 million square km (13.1 million square miles) which was about 23% of the world's total land area at the time.\n\nSo while the Mongol Empire was incredibly vast for its time, the British Empire was slightly larger in total area at its zenith. However, it's important to note that landmass comparisons don't fully capture other aspects of imperial power and control.\n\nIn summary:\n- Peak Mongol Empire area: Approximately 33 million sq km\n- Peak 

In [80]:
class FakeDatabase:
    def __init__(self):
        self.customers = [
            {"id": "1213210", "name": "John Doe", "email": "john@gmail.com", "phone": "123-456-7890", "username": "johndoe"},
            {"id": "2837622", "name": "Priya Patel", "email": "priya@candy.com", "phone": "987-654-3210", "username": "priya123"},
            {"id": "3924156", "name": "Liam Nguyen", "email": "lnguyen@yahoo.com", "phone": "555-123-4567", "username": "liamn"},
            {"id": "4782901", "name": "Aaliyah Davis", "email": "aaliyahd@hotmail.com", "phone": "111-222-3333", "username": "adavis"},
            {"id": "5190753", "name": "Hiroshi Nakamura", "email": "hiroshi@gmail.com", "phone": "444-555-6666", "username": "hiroshin"},
            {"id": "6824095", "name": "Fatima Ahmed", "email": "fatimaa@outlook.com", "phone": "777-888-9999", "username": "fatimaahmed"},
            {"id": "7135680", "name": "Alejandro Rodriguez", "email": "arodriguez@protonmail.com", "phone": "222-333-4444", "username": "alexr"},
            {"id": "8259147", "name": "Megan Anderson", "email": "megana@gmail.com", "phone": "666-777-8888", "username": "manderson"},
            {"id": "9603481", "name": "Kwame Osei", "email": "kwameo@yahoo.com", "phone": "999-000-1111", "username": "kwameo"},
            {"id": "1057426", "name": "Mei Lin", "email": "meilin@gmail.com", "phone": "333-444-5555", "username": "mlin"}
        ]

        self.orders = [
            {"id": "24601", "customer_id": "1213210", "product": "Wireless Headphones", "quantity": 1, "price": 79.99, "status": "Shipped"},
            {"id": "13579", "customer_id": "1213210", "product": "Smartphone Case", "quantity": 2, "price": 19.99, "status": "Processing"},
            {"id": "97531", "customer_id": "2837622", "product": "Bluetooth Speaker", "quantity": 1, "price": "49.99", "status": "Shipped"}, 
            {"id": "86420", "customer_id": "3924156", "product": "Fitness Tracker", "quantity": 1, "price": 129.99, "status": "Delivered"},
            {"id": "54321", "customer_id": "4782901", "product": "Laptop Sleeve", "quantity": 3, "price": 24.99, "status": "Shipped"},
            {"id": "19283", "customer_id": "5190753", "product": "Wireless Mouse", "quantity": 1, "price": 34.99, "status": "Processing"},
            {"id": "74651", "customer_id": "6824095", "product": "Gaming Keyboard", "quantity": 1, "price": 89.99, "status": "Delivered"},
            {"id": "30298", "customer_id": "7135680", "product": "Portable Charger", "quantity": 2, "price": 29.99, "status": "Shipped"},
            {"id": "47652", "customer_id": "8259147", "product": "Smartwatch", "quantity": 1, "price": 199.99, "status": "Processing"},
            {"id": "61984", "customer_id": "9603481", "product": "Noise-Cancelling Headphones", "quantity": 1, "price": 149.99, "status": "Shipped"},
            {"id": "58243", "customer_id": "1057426", "product": "Wireless Earbuds", "quantity": 2, "price": 99.99, "status": "Delivered"},
            {"id": "90357", "customer_id": "1213210", "product": "Smartphone Case", "quantity": 1, "price": 19.99, "status": "Shipped"},
            {"id": "28164", "customer_id": "2837622", "product": "Wireless Headphones", "quantity": 2, "price": 79.99, "status": "Processing"}
        ]

    def get_user(self, key, value):
        if key in {"email", "phone", "username"}:
            for customer in self.customers:
                if customer[key] == value:
                    return customer
            return f"Couldn't find a user with {key} of {value}"
        else:
            raise ValueError(f"Invalid key: {key}")
        
        return None

    def get_order_by_id(self, order_id):
        for order in self.orders:
            if order["id"] == order_id:
                return order
        return None
    
    def get_customer_orders(self, customer_id):
        return [order for order in self.orders if order["customer_id"] == customer_id]

    def cancel_order(self, order_id):
        order = self.get_order_by_id(order_id)
        if order:
            if order["status"] == "Processing":
                order["status"] = "Cancelled"
                return "Cancelled the order"
            else:
                return "Order has already shipped.  Can't cancel it."
        return "Can't find that order!"

In [81]:
db = FakeDatabase()

In [82]:
db.get_user("email", "john@gmail.com")


{'id': '1213210',
 'name': 'John Doe',
 'email': 'john@gmail.com',
 'phone': '123-456-7890',
 'username': 'johndoe'}

In [83]:
db.get_user("username", "adavis")

{'id': '4782901',
 'name': 'Aaliyah Davis',
 'email': 'aaliyahd@hotmail.com',
 'phone': '111-222-3333',
 'username': 'adavis'}

In [84]:
db.get_user("phone", "666-777-8888")

{'id': '8259147',
 'name': 'Megan Anderson',
 'email': 'megana@gmail.com',
 'phone': '666-777-8888',
 'username': 'manderson'}

In [85]:
db.get_customer_orders("1213210")

[{'id': '24601',
  'customer_id': '1213210',
  'product': 'Wireless Headphones',
  'quantity': 1,
  'price': 79.99,
  'status': 'Shipped'},
 {'id': '13579',
  'customer_id': '1213210',
  'product': 'Smartphone Case',
  'quantity': 2,
  'price': 19.99,
  'status': 'Processing'},
 {'id': '90357',
  'customer_id': '1213210',
  'product': 'Smartphone Case',
  'quantity': 1,
  'price': 19.99,
  'status': 'Shipped'}]

In [86]:
db.get_customer_orders("9603481")

[{'id': '61984',
  'customer_id': '9603481',
  'product': 'Noise-Cancelling Headphones',
  'quantity': 1,
  'price': 149.99,
  'status': 'Shipped'}]

In [87]:
db.get_order_by_id('24601')

{'id': '24601',
 'customer_id': '1213210',
 'product': 'Wireless Headphones',
 'quantity': 1,
 'price': 79.99,
 'status': 'Shipped'}

In [88]:
db.get_order_by_id("47652")

{'id': '47652',
 'customer_id': '8259147',
 'product': 'Smartwatch',
 'quantity': 1,
 'price': 199.99,
 'status': 'Processing'}

In [89]:
db.cancel_order("47652")

'Cancelled the order'

In [90]:
db.get_order_by_id("47652")

{'id': '47652',
 'customer_id': '8259147',
 'product': 'Smartwatch',
 'quantity': 1,
 'price': 199.99,
 'status': 'Cancelled'}

In [96]:
db_tool_schemas = [
    {
        "name": "get_user",
        "description": "Get the full user information based on their email, phone or username",
        "input_schema": {
            "type": "object",
            "properties": {
                "key": {
                    "type": "string",
                    "enum": ["email", "phone", "username"],
                    "description": "The specific key field - email, phone or username - being used to retrieve the full user information"
                },
                "value": {
                    "type": "string",
                    "description": "The value of the key field for the user whose information is being retrieved."
                }
            },
            "required": ["key", "value"],
        },
    },
    {
        "name": "get_order_by_id",
        "description": "Retrieves the information for a customer order based on the order id",
        "input_schema": {
            "type": "object",
            "properties": {"order_id": {"type": "string", "description": "The order ID"}},
            "required": ["order_id"],
        },
    },
    {
        "name": "get_customer_orders",
        "description": "Get all orders from a given customer",
        "input_schema": {
            "type": "object",
            "properties": {"customer_id": {"type": "string", "description": "The customer ID for the customer whose orders are being retrieved"}},
            "required": ["customer_id"],
        },
    },
    {
        "name": "cancel_order",
        "description": "Cancels the specified order",
        "input_schema": {
            "type": "object",
            "properties": {"order_id": {"type": "string", "description": "The order ID"}},
            "required": ["order_id"],
        },
    }
]
TOOL_MAP["get_user"] = db.get_user
TOOL_MAP["get_order_by_id"] = db.get_user
TOOL_MAP["get_customer_orders"] = db.get_customer_orders
TOOL_MAP["cancel_order"] = db.cancel_order